In [ ]:
1. Entity Relationship Diagram (ERD) for Online Purchasing Database
Entities:

Users (UserID, Name, Email, Address)

Products (ProductID, Name, Price, Stock)

Orders (OrderID, UserID, OrderDate, TotalAmount)

OrderDetails (OrderDetailID, OrderID, ProductID, Quantity)

Relationships:

Users → Orders (1-to-Many: One user can place many orders).

Orders → OrderDetails (1-to-Many: One order has multiple products).

Products → OrderDetails (1-to-Many: A product can appear in many orders).

In [ ]:
2. Create Database Objects
sql

-- Users Table
CREATE TABLE Users (
    UserID INT PRIMARY KEY AUTO_INCREMENT,
    Name VARCHAR(100) NOT NULL,
    Email VARCHAR(100) UNIQUE NOT NULL,
    Address VARCHAR(200)
);

-- Products Table
CREATE TABLE Products (
    ProductID INT PRIMARY KEY AUTO_INCREMENT,
    Name VARCHAR(100) NOT NULL,
    Price DECIMAL(10,2) NOT NULL,
    Stock INT DEFAULT 0
);

-- Orders Table
CREATE TABLE Orders (
    OrderID INT PRIMARY KEY AUTO_INCREMENT,
    UserID INT,
    OrderDate DATE DEFAULT CURRENT_DATE,
    TotalAmount DECIMAL(10,2),
    FOREIGN KEY (UserID) REFERENCES Users(UserID)
);

-- OrderDetails Table
CREATE TABLE OrderDetails (
    OrderDetailID INT PRIMARY KEY AUTO_INCREMENT,
    OrderID INT,
    ProductID INT,
    Quantity INT NOT NULL,
    FOREIGN KEY (OrderID) REFERENCES Orders(OrderID),
    FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
);

In [ ]:
3. Stored Procedure for User Registration with Validation
DELIMITER $$
CREATE PROCEDURE RegisterUser(
    IN p_Name VARCHAR(100),
    IN p_Email VARCHAR(100),
    IN p_Address VARCHAR(200)
)
BEGIN
    DECLARE EXIT HANDLER FOR SQLEXCEPTION
    BEGIN
        ROLLBACK;
        SELECT 'Error: Transaction rolled back' AS Result;
    END;

    START TRANSACTION;
    
    -- Validate email format
    IF p_Email NOT LIKE '%@%.%' THEN
        SIGNAL SQLSTATE '45000' SET MESSAGE_TEXT = 'Invalid email format';
    END IF;

    -- Check if email exists
    IF EXISTS (SELECT 1 FROM Users WHERE Email = p_Email) THEN
        SIGNAL SQLSTATE '45000' SET MESSAGE_TEXT = 'Email already registered';
    END IF;

    -- Insert new user
    INSERT INTO Users (Name, Email, Address) 
    VALUES (p_Name, p_Email, p_Address);

    COMMIT;
    SELECT 'User registered successfully' AS Result;
END$$
DELIMITER ;

Usage:

CALL RegisterUser('Alice Smith', 'alice@example.com', '123 Main St');

In [ ]:
4. SQL Aggregate Functions
Common aggregate functions:

COUNT(): Count rows.

SUM(): Sum values.

AVG(): Calculate average.

MIN()/MAX(): Find smallest/largest value.
    SELECT 
    COUNT(OrderID) AS TotalOrders,
    SUM(TotalAmount) AS TotalRevenue,
    AVG(TotalAmount) AS AverageOrderValue
FROM Orders;

In [ ]:
5. Pivot Query in SQL
Convert rows to columns (e.g., total sales per product by month):

SELECT 
    ProductID,
    SUM(CASE WHEN MONTH(OrderDate) = 1 THEN Quantity ELSE 0 END) AS Jan_Sales,
    SUM(CASE WHEN MONTH(OrderDate) = 2 THEN Quantity ELSE 0 END) AS Feb_Sales
FROM OrderDetails
JOIN Orders USING (OrderID)
GROUP BY ProductID;

In [ ]:
6. SQL Joins with Example
Inner Join: Retrieve orders with user details.
SELECT 
    Users.Name, 
    Orders.OrderID, 
    Orders.OrderDate
FROM Users
INNER JOIN Orders 
    ON Users.UserID = Orders.UserID;
Types of Joins:

INNER JOIN: Matched rows from both tables.

LEFT JOIN: All rows from the left table + matched rows from the right.

RIGHT JOIN: Opposite of LEFT JOIN.

In [ ]:
7. Find the 4th Highest Value in a Column
Step 1: Create a sample table:
CREATE TABLE Employees (
    EmployeeID INT PRIMARY KEY,
    Name VARCHAR(100),
    Salary DECIMAL(10,2)
);

INSERT INTO Employees VALUES 
(1, 'Alice', 60000),
(2, 'Bob', 75000),
(3, 'Charlie', 90000),
(4, 'David', 85000),
(5, 'Eve', 95000);
Step 2: Query for the 4th highest salary:
SELECT DISTINCT Salary
FROM Employees
ORDER BY Salary DESC
LIMIT 1 OFFSET 3;
Result:

Salary
75000